In [ ]:
import pandas as pd

df_train = pd.read_csv('/content/drive/MyDrive/PosPuc/TCC/datasets/df_processeds/x_train.csv')
y_train = pd.read_csv('/content/drive/MyDrive/PosPuc/TCC/datasets/df_processeds/y_train.csv')

df_test = pd.read_csv('/content/drive/MyDrive/PosPuc/TCC/datasets/df_processeds/x_test.csv')
y_test = pd.read_csv('/content/drive/MyDrive/PosPuc/TCC/datasets/df_processeds/y_test.csv')

In [ ]:
# Encoding variables
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore').fit(df_train)

# Encode train dataset
x_train = ohe.transform(df_train).toarray()
x_test = ohe.transform(df_test).toarray()

In [ ]:
# Change y_train values to float
y_train = y_train.astype('float32')

In [ ]:
# Checking GPU use
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found')
else:
    print('Found GPU at: {}'.format(device_name))
#   raise SystemError('GPU device not found')

GPU device not found


In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam, SGD


def built_model(layers_size, layers, optimizer, activation):
    
    hidden = []

    hidden.append(Input(shape=(x_train.shape[1],)))
    for _ in range(layers):
        hidden.append(Dense(layers_size, activation=activation))
        hidden.append(Dropout(0.2))
    
    hidden += [Dense(layers_size // 2, activation=activation), Dropout(0.2), Dense(1)]

    model = Sequential(hidden)
    
    model.compile(
        loss='mse', 
        optimizer=optimizer,
        metrics=[tf.keras.metrics.MeanAbsoluteError()])

    return model

In [ ]:
# Grid to search bests hyperparameters
results = []

optimizers = [SGD()]
layers_size = [32, 64, 128]
layers = [2, 3, 4]
epochs = [100, 200, 300]
activation = ['relu', 'gelu']

tf.random.set_seed(42)

for opt in optimizers:
    for lsize in layers_size:
        for l in layers:
            for act in activation:
                model = built_model(lsize, l, opt, act)
                for eph in epochs:
                    model.fit(x_train, y_train.values.ravel(), epochs=eph, verbose=False)
                    results.append(
                        {'optimizer': opt.get_config()['name'], 
                         'layers_size': lsize, 
                         'layers': l, 
                         'epochs': eph, 
                         'activation': act, 
                         'loss': model.history.history['loss'][-1] if not tf.math.is_nan(model.history.history['loss'][-1]) else 'nan', 
                         'mae': model.history.history['mean_absolute_error'][-1] if not tf.math.is_nan(model.history.history['mean_absolute_error'][-1]) else 'nan'})
                    results_nn = pd.DataFrame(results)
                    results_nn.to_csv('/content/drive/MyDrive/PosPuc/TCC/datasets/acidentes_ocorrencia/results_sgd.csv', index=False, encoding='utf8')

In [ ]:
Results
results_nn = pd.DataFrame(results)
results_nn.to_csv('/content/drive/MyDrive/PosPuc/TCC/datasets/df_processeds/results_nn.csv', index=False, encoding='utf8')